### 네이버 카페 크롤러 

##### 1.필요한 라이브러리 

In [1]:
# from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait # 로딩까지 기다리는
from selenium.webdriver.support import expected_conditions as EC # #예외상황이 떴을 때 대처하는 라이브러리
from selenium.webdriver.chrome.options import Options #필요한 옵션을 가져옴

from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys #키보드
from selenium.webdriver.common.by import By
from selenium import webdriver
from bs4 import BeautifulSoup as bs
from tqdm import tqdm
import pandas as pd
import pyperclip
import time
import csv
import random

In [103]:
# 차이점은??
# from selenium.webdriver.chrome.options import Options #필요한 옵션을 가져옴
# options = webdriver.ChromeOptions()

In [2]:
# 페이지 이동
def move(url):
    driver.get(url)
    driver.implicitly_wait(2)

# 로그인 입력
def login(id, pw):
    driver.execute_script("document.getElementsByName('dmswjd_14')[0].value = \'" + id + "\'")
    driver.execute_script("document.getElementsByName('1q2w3e1004!')[0].value = \'" + pw + "\'")
    time.sleep(1)
    # 로그인 클릭
    driver.find_element(By.ID, 'log.login').click()
    time.sleep(6)

# 카페 검색창에 검색
def search(item):
    pyperclip.copy(item)
    search = driver.find_element(By.NAME, "query")
    search.click()
    search.send_keys(Keys.CONTROL, 'v')
    driver.find_element(By.XPATH, '//*[@id="cafe-search"]/form/button').click()
    driver.implicitly_wait(5)


In [3]:
# 크롬드라이버 제어
chrome_options = Options()
chrome_options.add_argument('--no-sandbox') 
chrome_options.add_argument('--disable-dex-shm-usage')
driver = webdriver.Chrome(options=chrome_options, executable_path=r"C:\Developes\python\고객분석\chromedriver.exe")

# 1. 로그인하기
login_url = "https://nid.naver.com/nidlogin.login"

# 1-1. 로그인 페이지로 이동
move(login_url)

# 1-2. 아이디와 비밀번호 입력하기
login('dmswjd_14', '1q2w3e1004!'); time.sleep(5)

# 2. 크롤링하기



In [4]:
# # 2-1. 원하는 게시판으로 이동
current_url = 'https://cafe.naver.com/dogpalza?iframe_url_utf8=%2FArticleRead.nhn%253Fclubid%3D10258021%2526articleid%3D17847601%2526commentFocus%3Dtrue'
move(current_url)

In [5]:
# 제목, 본문, 댓글, 시간

title_list = []
content_list = []
comment_list = []
date_list = []

count_view = 0

# # 첫번째 게시물 클릭
driver.switch_to.frame("main-area")
driver.find_element(By.XPATH, '//*[@id="main-area"]/div[4]/table/tbody/tr[1]/td[1]/div[2]/div/a').click()


# 크롤링 (for loop . 지정한 게시물 개수까지)
for i in tqdm(range(2)):  # New tab
    for i in tqdm(range(30)):  # 한번에 몇 개의 게시물을 크롤링 (1 X 30 = 30)
        driver.implicitly_wait(5)
        time.sleep(2)

        # title
        time.sleep(random.uniform(0.5, 1))  # 0.5 ~ 1 sec
        title = driver.find_element(By.XPATH, '//*[@id="app"]/div/div/div[2]/div[1]/div[1]/div/h3').text
        title_list.append(title)

        # content (to prevent 3 case of errors)
        try:
            content = driver.find_element(
                By.CSS_SELECTOR, '#app > div > div > div.ArticleContentBox > div.article_container > div.article_viewer > div').text
            content_list.append(content)
        except:
            try:
                content = driver.find_element(By.CSS_SELECTOR, 'div.ContentRenderer').text
            except:
                content_pic = []  # 본문이 사진으로만 구성된 경우
                content_list.append(content_pic)
# 댓글
        # comment (With comment or without comment)
        # Retrieving html info of current page
        soup = bs(driver.page_source, 'lxml')
        iscomment = soup.find_all('span', class_= 'text_comment')
        if len(iscomment) == 0:  # Without Comment
            box = []
        
        else:
            WebDriverWait(driver, 15).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'text_comment')))
            
            soup = bs(driver.page_source,'lxml')
            iscomment = soup.find_all('span', class_= 'text_comment')
            box = []
            for i in iscomment:
                box.append([i.text])
        comment_list.append(box)
        
        # date
        time.sleep(random.uniform(0.5, 1))  # 0.5 ~ 1 sec
        times = driver.find_element(By.XPATH,'//*[@id="app"]/div/div/div[2]/div[1]/div[2]/div/div[2]/span[1]').text
        date_list.append(times)
        
        # Count_view (Crawling process)
        count_view = count_view + 1
        if count_view % 50 == 0: #50으로 나누었을 때 0, 크롤링 완료
            print('{} of contents were crawled successfully.'.format(count_view))
        else:
            pass
        
        # '다음 버튼'
        try:
            driver.find_element(By.CSS_SELECTOR, '#app > div > div > div.ArticleTopBtns > div.right_area > a.BaseButton.btn_next.BaseButton--skinGray.size_default > span').click()
            driver.implicitly_wait(28)
        except:
            current_url = driver.current_url
            driver.close()
            
            # 크롬창 다시 열기
            driver = webdriver.Chrome(options=chrome_options, executable_path=r"C:\Developes\python\고객분석\chromedriver.exe")
            
            # 멈ㅊㅜㄴ 부분 링크
            driver.get(current_url)
            driver.switch_to.frame('cafe_main')
            time.sleep(3)
            
            # 재로그인
            move(login_url)
            login('dmswjd_14', '1q2w3e1004!'); time.sleep(5)
            
            # 다음 링크부터 가져오기
            driver.get(current_url)
            driver.switch_to.frame('cafe_main')
            time.sleep(3)
        
    # 크롬이 다시 열릴 때 가지고 올 다음 글 링크 (href 추출)
    link = driver.find_element(By.XPATH,'//*[@id="app"]/div/div/div[1]/div[2]/a[1]').get_attribute('href')

    # 현재 크롬창 닫기
    driver.close()

    # 크롬창 다시 열기
    driver = webdriver.Chrome(options=chrome_options, executable_path=r"C:\Developes\python\고객분석\chromedriver.exe")

    # 재로그인
    move(login_url)
    login('아이디', '비번'); time.sleep(0.5)
    time.sleep(5)

    # 다음 링크부터 가져오기
    driver.get(link)
    driver.switch_to.frame('cafe_main')

#크롬창 끄기
driver.quit()

  0%|          | 0/2 [00:44<?, ?it/s]


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=107.0.5304.107)


In [111]:
print(len(title_list))
print(title_list)
print()
print(len(content_list))
print(content_list)
print()
print(len(comment_list))
print(comment_list)
print()
print(len(date_list))
print(date_list)

30
['강박, 집착 치매때문에 생기는걸까요?', '어머니가 집을 못 찾아오셔서 걱정입니다', '치매이야기와 함께하는★★★제7회 치매아카데미★★★', '요즘 휴머니튜드 책을 읽고 있는데', '마산이나 부산 병원추천부탁드려요', '노인장기요양등급', '제목 그되로입니다', '요즘 많이 힘드네요', '너무 화가나서', '너무 힘드네요', '병원처방 치매 약을 위산과다 약으로 처방한 약사 무혐의 처분', '어머니 치매검사 하고싶은데 혹시 수원에 어느병원에서 하면 좋을까요?', '치매 어머니, 요양병원에 안보내고...서울대 교수의 307일', '“치매 돌보는 사람도 치료해야 효과”… ‘보호자 케어’ 개발 선구자', '할아버지가 치매로 걱정이 많습니다.', '치매가족의 경제적 부담', '환자에게 치매사실 알리기', '망상·환각·공격성 - 약물, 비약물 치료로 조절', '치매환자와 의사소통을 촉진하기 위한 방법', '치매가족의 스트레스 관리', '친정아부지', '난, 우울하거나 슬프지 않습니다.', '[공유] 치매소재 영화, 스틸 앨리스(Still Alice)', '지금이 행복한, 나는 조금숙입니다.', '[공유] 치매환자 실금 문제와 화장실 이용하기', '[공유] 치매환자 혼자 두는 경우', '[공유] 치매 가족의 스트레스 관리', '우울증이 치매로 발전한 남성의 사례', '[공유] 부부 동반 치매 소재의 영화 [로망]', '치매에 걸린 엄마는 딸과의 추억을 기억할까요?']

30
['아빠가 치매 중증이신데,,\n6~7년 전에도 치매 초기라고 병원에서 진단받았는데,, 아빠가 정신병원에 다시는 안간다고 완강히 거부해서\n치료를 하지 못햇습니다~\n올해 갑자기 증세가 심해져서 병원에 갓는데 중증이라고 해서 약 복욕 중이세요.\n거동은 하시는데,, 가끔씩 예민해져서 일상생활에 가족들이 힘드네요..\n소리에 많이 예민하시고, 빨래에 집착하세요 ㅠ\n예전에도 큰소리 나는거 싫어하시고, 빨래도 본인이 건조대에 직접 널고 하셧거든요..\n근데 이부분이 유난히 더 심한 강박?집착이 

In [114]:
df = pd.DataFrame({"title_list":title_list, 'content_list':content_list, "comment_list": comment_list, "date_list":date_list})
df.head(3)
df.to_excel("치매가족이야기.xlsx", encoding="utf-8-sig", index=False)